In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
RANDOM_SEED = 42
os.environ["TF_USE_LEGACY_KERAS"] = "1"


# Specify each path

In [2]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.keras'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [3]:
NUM_CLASSES = 36

# Dataset reading

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [8]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 36)                396       
                                                                 
Total params: 1466 (5.73 KB)
Trainable params: 1466 (5.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [10]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [11]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/20 [>.............................] - ETA: 2:01 - loss: 3.5791 - accuracy: 0.0156
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.keras
20/20 [==============================] - 7s 13ms/step - loss: 3.5569 - accuracy: 0.0364 - val_loss: 3.5019 - val_accuracy: 0.0683
Epoch 2/1000
 1/20 [>.............................] - ETA: 0s - loss: 3.5305 - accuracy: 0.0391
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.keras
20/20 [==============================] - 0s 2ms/step - loss: 3.5036 - accuracy: 0.0536 - val_loss: 3.4341 - val_accuracy: 0.1401
Epoch 3/1000
 1/20 [>.............................] - ETA: 0s - loss: 3.4632 - accuracy: 0.0703
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.keras
20/20 [==============================] - 0s 2ms/step - loss: 3.4528 - accuracy: 0.0735 - val_loss: 3.3597 - val_accuracy: 0.1234
Epoch 4/1000
 1/20 [>.............................] - ETA: 0s - loss: 3.4366 - accuracy: 0.

In [12]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

7/7 [==============================] - 0s 708us/step - loss: 0.8172 - accuracy: 0.8695


In [13]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [14]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 213ms/step
[4.03740487e-05 6.53708266e-05 9.76693988e-01 7.79437178e-05
 3.04898713e-04 5.58694546e-06 2.17299323e-13 3.98724543e-12
 1.02776615e-02 3.07772099e-03 5.22552534e-19 1.56054973e-08
 1.53567645e-11 1.33466187e-07 9.37165692e-03 1.82144192e-16
 1.23444917e-12 2.98832831e-18 5.86107551e-13 3.26005935e-20
 3.56897706e-10 1.29348643e-09 1.96846434e-16 4.79342989e-11
 3.99197415e-05 3.65008896e-06 2.31777352e-23 1.70166663e-32
 1.77409941e-29 1.45896912e-26 2.89204233e-22 3.36119881e-27
 7.82510421e-27 3.82439658e-30 5.20126440e-26 4.10456887e-05]
2


# Confusion matrix

# Convert to model for Tensorflow-Lite

In [15]:
# Save as a model dedicated to inference
model.save(model_save_path)

In [16]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: /var/folders/5t/g29mwtp54hxb07k8g6pwpxvr0000gn/T/tmp9_b083qg/assets


INFO:tensorflow:Assets written to: /var/folders/5t/g29mwtp54hxb07k8g6pwpxvr0000gn/T/tmp9_b083qg/assets
W0000 00:00:1715900221.232946 107968169 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1715900221.232960 107968169 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.


7848

# Inference test

In [17]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [18]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [19]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [20]:

# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [21]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[4.0374012e-05 6.5370637e-05 9.7669399e-01 7.7943572e-05 3.0489816e-04
 5.5869400e-06 2.1730016e-13 3.9872688e-12 1.0277642e-02 3.0777254e-03
 5.2255460e-19 1.5605439e-08 1.5356766e-11 1.3346595e-07 9.3716616e-03
 1.8214492e-16 1.2344541e-12 2.9883401e-18 5.8610761e-13 3.2600351e-20
 3.5689707e-10 1.2934865e-09 1.9684721e-16 4.7934209e-11 3.9919782e-05
 3.6500821e-06 2.3177827e-23 1.7016540e-32 1.7740997e-29 1.4589805e-26
 2.8920428e-22 3.3612123e-27 7.8251643e-27 3.8244263e-30 5.2012847e-26
 4.1045652e-05]
2
